In [ ]:
from generative_ai import GenAIModel
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from dotenv import dotenv_values
import asyncio
import time
import re

In [ ]:
async def get_channel(session_name, api_id, api_hash, phone_number, channel_username):
    client = TelegramClient(session_name, api_id, api_hash)
    await client.start(phone_number)

    return await client.get_entity(channel_username)

In [ ]:
async def get_history(channel, offset_id, limit, timeout:float = None):
    
    start_time = time.time()
    all_messages = []
    
    while True:
        history = await(GetHistoryRequest(
            peer=channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))

        if not history.messages:
            break

        all_messages.extend(history.messages)
        offset_id = all_messages[-1].id

        if timeout is not None and timeout < time.time() - start_time:
            break

    return all_messages

In [ ]:
async def model_ask_iterator(model:GenAIModel, iterable:list[str], query:str, timeout:float = None):
    
    start_time = time.time()
    responses = []
    last = 0
    
    while(last < len(iterable)):
        for i, current in enumerate(iterable[last:]):
            print(f"Current element: {i}, elements left: {len(iterable) - i}")
            try:
                responses.append(model.ask_model(query + current))
            except:
                break
        
        last = len(responses)
        time.sleep(10)

        if timeout is not None and timeout < time.time() - start_time:
            break

    return responses

In [ ]:
def generate_code(filename:str, iterable:list[str], before_iterations:str="", during_iterations:str="", after_iterations:str=""):

    with open(filename, "w") as file:

        file.write(before_iterations)
        for current in iterable:
            file.write(current)
            file.write(during_iterations)
        file.write(after_iterations)

# Este fragmento de código debe utilizarse con la ubicación correspondiente al número de teléfono utilizado

In [ ]:
secrets = dotenv_values(".env")

TELEGRAM_API_ID = secrets["TELEGRAM_API_ID"]
TELEGRAM_API_HASH = secrets["TELEGRAM_API_HASH"]
TELEGRAM_PHONE_NUMBER = secrets["TELEGRAM_PHONE_NUMBER"]
CHANNEL_USERNAME = "EmpresaElectricaDeLaHabana"

channel = get_channel("Scrapper", TELEGRAM_API_ID, TELEGRAM_API_HASH, TELEGRAM_PHONE_NUMBER)

# A partir de este punto es necesario el uso de VPN si se ejecuta en una ubicación no permitido por la API de Gemini

In [ ]:
model = GenAIModel

telegram_history = [message for message in get_history(channel, 0, 100) if "Nota" in message]

In [ ]:
query = "extrae la fecha, las unidades que salen de circulación con la cantidad de MW que producen, las unidades que entran en circulación con sus respectivos MW, la generación total, la generación esperada y lo que se espera que ocurra en el pico del siguiente mensaje:\n"

extracted_info = model_ask_iterator(model, telegram_history, query)

In [ ]:
class_info = """
class InformeGeneracion:
    def __init__(self, fecha, unidades_fuera, unidades_entrando, generacion_total,
                 generacion_esperada, demanda_actual, demanda_estimada, afectacion_pico,
                 informacion_adicional=None):
        self.fecha = fecha
        self.unidades_fuera = unidades_fuera
        self.unidades_entrando = unidades_entrando
        self.generacion_total = generacion_total
        self.generacion_esperada = generacion_esperada
        self.demanda_actual = demanda_actual
        self.demanda_estimada = demanda_estimada
        self.afectacion_pico = afectacion_pico
        self.informacion_adicional = informacion_adicional   
"""

example = """
# Extracción y asignación de valores del texto a la variable 'instance'
instance = InformeGeneracion(
    fecha=(25, 9, 2024),
    unidades_fuera=[
        "CTE Felton: Unidad 2",
        "CTE Renté: Unidad 5",
        "CTE Santa Cruz: Unidad 2",
        "46 centrales de generación distribuida",
        "Generación Móvil de Santiago de Cuba",
        "Motores en la Generación Móvil de Melones"
    ],
    unidades_entrando=[
        "5 motores de la Generación Móvil de Melones"
    ],
    generacion_total=2025,
    generacion_esperada=2220,
    demanda_actual=2350,
    demanda_estimada=3000,
    afectacion_pico=850,
    informacion_adicional={
        "afectacion_media": 550,
        "limitaciones_termicas": 541,
        "recuperacion_generacion_distribuida": 80
    }
)
"""

explain = """
Genera en python solamente la asignacion a una variable llamada instance de una instancia de la clase anterior extrayendo la informacion correspondiente del siguiente texto donde la fecha es una tripla con la fecha en el formato DD-MM-AA solo con valores numericos, no utilices la libreria datetime, coloca directamente los valores correspondientes, unidades fuera contiente un array con los nombres de las unidades al igual que las unidades entrando (el formato es Termoelectrica: Unidad 1, Unidad 2, ...), y el resto de los valores contienen un valor numerico con la cantidad de MW correspondientes guiandote por el ejemplo anterior:
"""

query = f"Dada la clase {class_info} y el ejemplo {example} {explain}:\n"

class_instances = model_ask_iterator(model, extracted_info, query)

In [ ]:
regex = r"instance\s*=\s*InformeGeneracion.*?informacion_adicional.*?\)"

matches = re.findall(regex, class_instances)

In [ ]:
declarations = f"{class_info}\nclass_instances = []\n"
appends = "\nclass_instances.append(instance)\n"

generate_code("code.py", matches, before_iterations=declarations, during_iterations=appends)